In [1]:
# !pip install implicit

 Downloading implicit-0.6.1-cp39-cp39-manylinux2014_x86_64.whl (18.6 MB)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
data['week_no'].nunique()

95

In [5]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [6]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [7]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


### Train-test split

В рекомендательных системах корректнее использовать train-test split по времени, а не случайно  
Я возьму последние 3 недели в качестве теста

In [8]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [9]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [10]:
result = pd.read_csv('./data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [11]:
# проверка на наличие новых (холодных) пользователей
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

*Пример*  
item_1 - 5, item_2 - 7, item_3 - 4  # / sum  
item_1 - 5 / 16, item_2 - 7 / 16, item_3 - 4 / 16



def get_items_weights(df):
    sum_sales = df['sales_value'].sum()
    items_weights = df.groupby('item_id').agg({'sales_value':'sum'}).reset_index().rename(columns={'sales_value':'weight'})
    items_weights['weight']=items_weights['weight'].apply(lambda x: x/sum_sales)
    return items_weights

In [12]:
def weighted_random_recommendation(items_weights, n=5):
    """Взвешенные случайные рекомендации"""
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    
    # в параметр p задаём расчитанные веса товаров  
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [13]:
%%time

# создаем датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index().rename(columns={'sales_value':'weight'})
items_weights['weight'] = np.log(1 + items_weights['weight'])
items_weights['weight'] = items_weights['weight']/sum(items_weights['weight'])

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 1.72 s, sys: 4.58 ms, total: 1.72 s
Wall time: 1.72 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[860456, 1060677, 6391207, 873754, 891907]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[418697, 952122, 6463949, 986548, 1106401]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [14]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2042 entries, 0 to 2041
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   user_id                         2042 non-null   int64 
 1   actual                          2042 non-null   object
 2   random_recommendation           2042 non-null   object
 3   popular_recommendation          2042 non-null   object
 4   itemitem                        2042 non-null   object
 5   cosine                          2042 non-null   object
 6   tfidf                           2042 non-null   object
 7   own_purchases                   2042 non-null   object
 8   weighted_random_recommendation  2042 non-null   object
dtypes: int64(1), object(8)
memory usage: 143.7+ KB


In [15]:
# функция преобразует данные, чтобы их можно было в дальнейшем привести к числовому виду

def clear_list(data):
    if type(data) is str:
        result = []
        number = ''
        for i in data[1:-1]:
            if i != ' ' and i != ',' and i != '\n':
                number += i
            elif number != '':
                result.append(number)
                number = ''
        if number != '':
            result.append(number)
        return result
    return data

In [16]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(clear_list(bought_list)).astype(float)
    recommended_list = np.array(clear_list(recommended_list)[:k]).astype(float)
    
    flags = np.isin(recommended_list, bought_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [17]:
for col in result.columns[1:]:    
    print(f"{round(result.apply(lambda row: precision_at_k(row[col], row['actual'], k=5), axis=1).mean(),4)}:{col}")

1.0:actual
0.0006:random_recommendation
0.1552:popular_recommendation
0.0336:itemitem
0.0353:cosine
0.0361:tfidf
0.18:own_purchases
0.001:weighted_random_recommendation


#### Вывод: лучшее качество показывает алгоритм own_purchases, который предлагает пользователям товары, которые они уже покупали до этого.